## hw02

### 1.量价分析中应该深入考虑的是什么？

我觉得应该多考虑量价关系背后的多空力量的转化。

### 2.为什么放量上涨后的缩量回落是健康的？

股票走势是一种多空博弈。当多头制造一波上涨后，缩量回调代表空头力量较弱，所以是比较健康的。

### 3.请自己设置一个止盈与止损的操作策略。
当低位震荡时，静静观察；

当股价上涨突破时，平衡被打破，等待第一波上涨过后，如果股价缩量回调，建仓；

当股价创新高，量能没有创新高；或者量能创新高，股价没有创新高时，平仓。